<a href="https://colab.research.google.com/github/Inno732/Defi_IA_1_2023_2024/blob/main/XAI_Gradcam_classification_tf_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1. Intaller le module d'explicabilité "tf-explain"**

In [ ]:
!pip install tf-explain

#**2. Importer les libraries**

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import tf_explain
from tf_explain.core.grad_cam import GradCAM
from tf_explain.core.smoothgrad import SmoothGrad
from keras.models import load_model
import cv2

#**3. Télécharger la base d'images de test**

In [ ]:
!rm -rf sample_data
!wget https://nextcloud.ig.umons.ac.be/s/RsZpqtYfDgYzKxr/download/test.zip
!unzip test.zip
! rm test.zip

In [4]:
!printf '%s\n' 'fire' 'no_fire' 'start_fire'> classes.txt

In [5]:
dataset_name='test'
#test_dataset = os.path.join('bases/', dataset_name)
classes_path = "classes.txt"

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
	dataset_name,
	batch_size= 8,      # Taille du mini-batch
  label_mode= 'categorical'    # Conversion au format One-Hot
)                             # Generates a 'tf.data.Dataset' from image files in a directory (train_dataset)
print(test_ds.class_names)

#**4. Télécharger votre modèle**

In [ ]:
!wget https://nextcloud.ig.umons.ac.be/s/p2xNRJZqZ5zCrW6/download/small_MobileNetV2_2023_test.h5

In [ ]:
!wget https://nextcloud.ig.umons.ac.be/s/p2xNRJZqZ5zCrW6/download/model_VGG16_8r5eqnu.h5

In [ ]:
!wget https://nextcloud.ig.umons.ac.be/s/p2xNRJZqZ5zCrW6/download/model_VGG19_epD3OKv.h5
!wget https://nextcloud.ig.umons.ac.be/s/p2xNRJZqZ5zCrW6/download/model_rsnt50.h5
!wget https://nextcloud.ig.umons.ac.be/s/p2xNRJZqZ5zCrW6/download/model_xception_1.h5
!wget https://nextcloud.ig.umons.ac.be/s/p2xNRJZqZ5zCrW6/download/model_DsNt121_1.h5

In [10]:
# Load pretrained model or your own
model = load_model("small_MobileNetV2_2023_test.h5")

In [11]:
model_VGG16 = load_model("model_VGG16_8r5eqnu.h5")

In [12]:
model_VGG19 = load_model("model_VGG19_epD3OKv.h5")

In [13]:
model_DsNt121 = load_model("model_DsNt121_1.h5")

In [14]:
model_rsnt50 = load_model("model_rsnt50.h5")

In [15]:
model_xception = load_model("model_xception_1.h5")

In [16]:
input_dim =224

#**5. Définir la classe à expliquer**

In [17]:
className="Fire"
#className="start_fire"
!mkdir -p className/className

In [18]:
classes = test_ds.class_names

#**6. Sélectionner 3 images de test**

In [19]:
images_to_test = ["test/fire/test002.jpg",
                  "test/fire/test005.jpg",
                  "test/fire/test013.jpg",
                  "test/fire/F_2062.jpg",
                  "test/fire/F_2063.jpg",
                  "test/fire/F_2064.jpg",
                  "test/fire/F_2066.jpg",
                  "test/fire/F_2067.jpg",
                  "test/fire/F_2080.jpg",
                  "test/no_fire/img31.jpg",
                  "test/fire/F_2062.jpg",
                  "test/start_fire/test101.jpg",
                  "test/start_fire/test122.jpg",
                  "test/start_fire/NorthAmerica (79).png",
                  "test/start_fire/test106.jpg",
                  "test/start_fire/test118.jpg",
]

#**4. Expliquer le modèle avec la méthode XAI "GradCAM"**

In [ ]:
for image_path in images_to_test:
    #imnum=[]
    print(image_path)
    # Load to the correct format and predict the current image
    img0 = tf.keras.preprocessing.image.load_img(image_path, target_size=(input_dim,input_dim))
    img = tf.keras.preprocessing.image.img_to_array(img0)

    data = ([img], None)
    xy=np.expand_dims(img,axis=0)
    myTuple=(xy,None)

    # Start explainer
    #explainer = SmoothGrad()
    explainer = GradCAM()
    grid = explainer.explain(myTuple, model_VGG16, class_index=1)
    temp = np.concatenate((img,grid),axis=1)
    fig, axs = plt.subplots(1, 3)
    axs[0].imshow(img.astype(np.uint8))
    axs[0].set_title("input")
    axs[1].imshow(grid.astype(np.uint8))
    axs[1].set_title("XAI")

    fig.set_figheight(15)
    fig.set_figwidth(15)


    pred_VGG16 = model_VGG16.predict(xy,batch_size=1)[0]
    print(pred_VGG16)
    for (pos,prob) in enumerate(pred_VGG16):
      pos == np.argmax(pred_VGG16)
      img = cv2.imread(image_path)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      font = cv2.FONT_HERSHEY_COMPLEX
      textsize = cv2.getTextSize(className, font, 0.7, 2)[0]
      textX = (img.shape[1] - textsize[0]) / 2
      textY = (img.shape[0] + textsize[1]) / 2
      cv2.putText(img, classes[np.argmax(pred_VGG16)], (int(textX), int(textY)), font, 4.7, (255,255,255), 5, cv2.LINE_AA)
      axs[2].imshow(img)
    plt.show()

In [ ]:
from PIL import Image
%matplotlib inline

axes=[]
fig = plt.figure()
fig = plt.figure(figsize=(18,9))

classes = test_ds.class_names
liste_image = images_to_test
compteur = 1
for image_path in liste_image:
    # predict VGG16
    img = Image.open(image_path).convert('RGB')
    x = tf.keras.utils.img_to_array(img,data_format='channels_last')
    x = tf.keras.preprocessing.image.smart_resize(x, size=(299,299))
    x = np.expand_dims(x, axis=0)
    pred = model_VGG16.predict(x,batch_size=1)[0]


    for (pos,prob) in enumerate(pred):
            class_name = classes[pos]
            if (pos == np.argmax(pred)) :
                img = cv2.imread(image_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                font = cv2.FONT_HERSHEY_COMPLEX
                textsize = cv2.getTextSize(class_name, font, 1, 2)[0]
                textX = (img.shape[1] - textsize[0]) / 2
                textY = (img.shape[0] + textsize[1]) / 2
                cv2.putText(img, class_name, (int(textX)-10, int(textY)), font, 2, (255,0,0), 6, cv2.LINE_AA)
                axes.append(fig.add_subplot(4, 4, compteur))
                plt.imshow(img)
                compteur += 1
            # print("Top %d ====================" % (i + 1))
            print("Class Name : %s" % (classes[np.argmax(pred)]), "---", "Class Probability: %.2f%%" % (prob*100))

plt.show()

In [ ]:
for image_path in images_to_test:
    #imnum=[]
    print(image_path)
    # Load to the correct format and predict the current image
    img0 = tf.keras.preprocessing.image.load_img(image_path, target_size=(input_dim,input_dim))
    img = tf.keras.preprocessing.image.img_to_array(img0)

    data = ([img], None)
    xy=np.expand_dims(img,axis=0)
    myTuple=(xy,None)

    # Start explainer
    #explainer = SmoothGrad()
    explainer = GradCAM()
    grid = explainer.explain(myTuple, model, class_index=1)
    temp = np.concatenate((img,grid),axis=1)
    fig, axs = plt.subplots(1, 3)
    axs[0].imshow(img.astype(np.uint8))
    axs[0].set_title("input")
    axs[1].imshow(grid.astype(np.uint8))
    axs[1].set_title("XAI")

    fig.set_figheight(15)
    fig.set_figwidth(15)


    pred = model.predict(xy,batch_size=1)[0]
    print(pred)
    for (pos,prob) in enumerate(pred):
      pos == np.argmax(pred)
      img = cv2.imread(image_path)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      font = cv2.FONT_HERSHEY_COMPLEX
      textsize = cv2.getTextSize(className, font, 0.7, 2)[0]
      textX = (img.shape[1] - textsize[0]) / 2
      textY = (img.shape[0] + textsize[1]) / 2
      cv2.putText(img, classes[np.argmax(pred)], (int(textX), int(textY)), font, 4.7, (255,255,255), 5, cv2.LINE_AA)
      axs[2].imshow(img)
    plt.show()

In [ ]:
for image_path in images_to_test:
    #imnum=[]
    print(image_path)
    # Load to the correct format and predict the current image
    img0 = tf.keras.preprocessing.image.load_img(image_path, target_size=(input_dim,input_dim))
    img = tf.keras.preprocessing.image.img_to_array(img0)

    data = ([img], None)
    xy=np.expand_dims(img,axis=0)
    myTuple=(xy,None)

    # Start explainer
    #explainer = SmoothGrad()
    explainer = GradCAM()
    grid = explainer.explain(myTuple, model_VGG19, class_index=1)
    temp = np.concatenate((img,grid),axis=1)
    fig, axs = plt.subplots(1, 3)
    axs[0].imshow(img.astype(np.uint8))
    axs[0].set_title("input")
    axs[1].imshow(grid.astype(np.uint8))
    axs[1].set_title("XAI")

    fig.set_figheight(15)
    fig.set_figwidth(15)


    pred_VGG19 = model_VGG19.predict(xy,batch_size=1)[0]
    print(pred_VGG19)
    for (pos,prob) in enumerate(pred_VGG19):
      pos == np.argmax(pred_VGG19)
      img = cv2.imread(image_path)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      font = cv2.FONT_HERSHEY_COMPLEX
      textsize = cv2.getTextSize(className, font, 0.7, 2)[0]
      textX = (img.shape[1] - textsize[0]) / 2
      textY = (img.shape[0] + textsize[1]) / 2
      cv2.putText(img, classes[np.argmax(pred_VGG19)], (int(textX), int(textY)), font, 4.7, (255,255,255), 5, cv2.LINE_AA)
      axs[2].imshow(img)
    plt.show()

In [ ]:
for image_path in images_to_test:
    #imnum=[]
    print(image_path)
    # Load to the correct format and predict the current image
    img0 = tf.keras.preprocessing.image.load_img(image_path, target_size=(input_dim,input_dim))
    img = tf.keras.preprocessing.image.img_to_array(img0)

    data = ([img], None)
    xy=np.expand_dims(img,axis=0)
    myTuple=(xy,None)

    # Start explainer
    #explainer = SmoothGrad()
    explainer = GradCAM()
    grid = explainer.explain(myTuple, model_DsNt121, class_index=1)
    temp = np.concatenate((img,grid),axis=1)
    fig, axs = plt.subplots(1, 3)
    axs[0].imshow(img.astype(np.uint8))
    axs[0].set_title("input")
    axs[1].imshow(grid.astype(np.uint8))
    axs[1].set_title("XAI")

    fig.set_figheight(15)
    fig.set_figwidth(15)


    pred_DsNt121 = model_DsNt121.predict(xy,batch_size=1)[0]
    print(pred_DsNt121)
    for (pos,prob) in enumerate(pred_DsNt121):
      pos == np.argmax(pred_DsNt121)
      img = cv2.imread(image_path)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      font = cv2.FONT_HERSHEY_COMPLEX
      textsize = cv2.getTextSize(className, font, 0.7, 2)[0]
      textX = (img.shape[1] - textsize[0]) / 2
      textY = (img.shape[0] + textsize[1]) / 2
      cv2.putText(img, classes[np.argmax(pred_DsNt121)], (int(textX), int(textY)), font, 4.7, (255,255,255), 5, cv2.LINE_AA)
      axs[2].imshow(img)
    plt.show()


In [ ]:
for image_path in images_to_test:
    #imnum=[]
    print(image_path)
    # Load to the correct format and predict the current image
    img0 = tf.keras.preprocessing.image.load_img(image_path, target_size=(input_dim,input_dim))
    img = tf.keras.preprocessing.image.img_to_array(img0)

    data = ([img], None)
    xy=np.expand_dims(img,axis=0)
    myTuple=(xy,None)

    # Start explainer
    #explainer = SmoothGrad()
    explainer = GradCAM()
    grid = explainer.explain(myTuple, model_xception, class_index=1)
    temp = np.concatenate((img,grid),axis=1)
    fig, axs = plt.subplots(1, 3)
    axs[0].imshow(img.astype(np.uint8))
    axs[0].set_title("input")
    axs[1].imshow(grid.astype(np.uint8))
    axs[1].set_title("XAI")

    fig.set_figheight(15)
    fig.set_figwidth(15)


    pred_xception = model_xception.predict(xy,batch_size=1)[0]
    print(pred_xception)
    for (pos,prob) in enumerate(pred_xception):
      pos == np.argmax(pred_xception)
      img = cv2.imread(image_path)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      font = cv2.FONT_HERSHEY_COMPLEX
      textsize = cv2.getTextSize(className, font, 0.7, 2)[0]
      textX = (img.shape[1] - textsize[0]) / 2
      textY = (img.shape[0] + textsize[1]) / 2
      cv2.putText(img, classes[np.argmax(pred_xception)], (int(textX), int(textY)), font, 4.7, (255,255,255), 5, cv2.LINE_AA)
      axs[2].imshow(img)
    plt.show()

In [ ]:
for image_path in images_to_test:
    #imnum=[]
    print(image_path)
    # Load to the correct format and predict the current image
    img0 = tf.keras.preprocessing.image.load_img(image_path, target_size=(input_dim,input_dim))
    img = tf.keras.preprocessing.image.img_to_array(img0)

    data = ([img], None)
    xy=np.expand_dims(img,axis=0)
    myTuple=(xy,None)

    # Start explainer
    #explainer = SmoothGrad()
    explainer = GradCAM()
    grid = explainer.explain(myTuple, model_rsnt50, class_index=1)
    temp = np.concatenate((img,grid),axis=1)
    fig, axs = plt.subplots(1, 3)
    axs[0].imshow(img.astype(np.uint8))
    axs[0].set_title("input")
    axs[1].imshow(grid.astype(np.uint8))
    axs[1].set_title("XAI")

    fig.set_figheight(15)
    fig.set_figwidth(15)


    pred_rsnt50 = model_rsnt50.predict(xy,batch_size=1)[0]
    print(pred_rsnt50)
    for (pos,prob) in enumerate(pred_rsnt50):
      pos == np.argmax(pred_rsnt50)
      img = cv2.imread(image_path)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      font = cv2.FONT_HERSHEY_COMPLEX
      textsize = cv2.getTextSize(className, font, 0.7, 2)[0]
      textX = (img.shape[1] - textsize[0]) / 2
      textY = (img.shape[0] + textsize[1]) / 2
      cv2.putText(img, classes[np.argmax(pred_rsnt50)], (int(textX), int(textY)), font, 4.7, (255,255,255), 5, cv2.LINE_AA)
      axs[2].imshow(img)
    plt.show()


In [ ]:
from PIL import Image
im = (Image.open('test/fire/F_2062.jpg'))
im